# <center> <u>**Extracting Paragraph and Marking various elements such as tables, images, headings, and lists present in  a PDF file**</u> </center>

In [ ]:
#Using the pretrained model layoutparser
!pip install -U layoutparser
#Since layout parser requires Detectron2 model for predicting, Detectron 2 must be installed
!pip install 'git+https://github.com/facebookresearch/detectron2.git@v0.4#egg=detectron2'
#Downlaoding ocr tool for extracting text from paragraph
!pip install layoutparser[ocr]
!sudo apt install tesseract-ocr
#Installing packages for converting pdf pages to Images
!apt-get install poppler-utils
!pip install pdf2image

Requirement already up-to-date: layoutparser in /usr/local/lib/python3.7/dist-packages (0.2.0)
Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
poppler-utils is already the newest version (0.62.0-2ubuntu2.12).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [ ]:
def plot_ele(page,model,element):
  """
  Takes image and model as argument and returns the location of bounding boxes for paragraphs and the process image in which paragraphs are marked
  """
  #Detecting all text, title, list, table  and images that are present in the image passed as argument
  layout=model.detect(page)
  #Filtering only paragraphs from the extracted content
  elements=lp.Layout([b for b in layout if b.type==element])
  #Drawing bounding boxes over the paragraph 
  fig=lp.draw_box(page, elements, box_width=3)
  return (elements,fig)

In [ ]:
def extract_text(image,paragraphs,extracted):
  """
  This function extracts the content present in the paragraphs.
  """
  for block in paragraphs:
    #padding the block of marked paragraph
    segment_image=(block.pad(left=5, right=5, top=5, bottom=5).crop_image(image))
    #extracting text using OCR    
    text=ocr_agent.detect(segment_image)
    block.set(text=text, inplace=True)
  #Storing all the accumulated text from paragraphs that are present in thle image
  for txt in paragraphs.get_texts():
    extracted+=txt
    extracted+='\n'
  return extracted 

In [ ]:
from pdf2image import convert_from_path
from PIL import Image
import numpy as np
import cv2
import layoutparser as lp
#Initializing the ocr agent
ocr_agent = lp.TesseractAgent(languages='eng')
label_map={0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"}
#Initializing any model that is a part of Layout parser 
model = lp.Detectron2LayoutModel('lp://PubLayNet/mask_rcnn_X_101_32x8d_FPN_3x/config',extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.65],
                                 label_map=label_map)

In [ ]:
#variable used to hold the extracted text from paragraph.
extracted=""
#Taking file name as input from user
file_path=input('Enter the complete location of the pdf file: ')
print("The elements such as\n 1. Paragraphs\n 2. Titles\n 3. Lists\n 4. Tables\n 5. Figures")
element=int(input("Enter the element to be extracted form the pdf: "))
if element>=1 and element<=5:
  element-=1
  #converting the given pdf into images
  pages=convert_from_path(file_path)
  arr=[]
  #looping through all the pages of the pdf and extracting the content bounded by rectangular bounding box
  for i in pages:
    i=np.asarray(i)
    elements,img=plot_ele(i,model,label_map[element])
    if element==0 or element==1:
      h, w = i.shape[:2]
      left_interval = lp.Interval(0, w/2*1.05, axis='x').put_on_canvas(i)
      left_blocks = elements.filter_by(left_interval, center=True)
      left_blocks.sort(key = lambda b:b.coordinates[1])
      right_blocks = [b for b in elements if b not in left_blocks]
      right_blocks.sort(key = lambda b:b.coordinates[1])
      elements = lp.Layout([b.set(id = idx) for idx, b in enumerate(left_blocks + right_blocks)])
      extracted=extract_text(i,elements,extracted)
    img=np.asarray(img)
    arr.append(img)
  arr=np.array(arr)
  result=cv2.vconcat(arr)
  result=Image.fromarray(result)
  result.save('result.pdf')
else:
  print("Please enter a valid element to be extracted.")

Enter the complete location of the pdf file: /content/pca_face.pdf
The elements such as
 1. Paragraphs
 2. Titles
 3. Lists
 4. Tables
 5. Figures
Enter the element to be extracted form the pdf: 1


In [ ]:
from google.colab import files
if element==0 or element==1:
  file=open('data.txt','w')
  file.write(extracted)
  file.close()
  files.download('data.txt')
files.download('result.pdf')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>